# FizzBuzz
FizzBuzz 是一个简单的小游戏。游记规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz。当遇到5的倍数的时候说Buzz ,当遇到3和5的倍数的时候说 FizzBuzz，其他时候是正常数。
我们可以通过一个简单的小程序来决定是要返回的是 fizz 还是 buzz 还是 fizzbuzz。

In [1]:
def fizz_buzz_encode(i):
    if i % 15 == 0: return 3
    if i %5 ==0: return 2
    if i % 3 ==0: return 1
    return 0
def fizz_buzz_decoding(i,prediction):
    return [str(i),'fizz','buzz','fizzbuzz'][prediction]
def helper(i):
    print(fizz_buzz_decoding(i,fizz_buzz_encode(i)))

for i in range(1,20):
    helper(i)


1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz
16
17
fizz
19


## 使用神经网络实现 fizzbuzz 的功能。
首先定义模型的输入与输出（训练数据）


In [2]:
import torch
import numpy as np

NUM_DIGITS = 10
# 将数字转换为二进制是为了增加输入，因为单独的一个数字作为输入相比于 二进制数做输入 ，二进制数的特征更多，这也算是特征工程的一种手段。
def binary_encode(i,num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)][::-1])

train_x = torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(101,2**NUM_DIGITS)])
train_y = torch.LongTensor([fizz_buzz_encode(i)for i in range(101,2**NUM_DIGITS)]) # 因为 y 代表的是类别所以不能用 float 类型的数值，改成Long


#### 我们用 PyTorch 定义模型

In [20]:
NUM_HIDDEN = 100

model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4) # after softmax, wo get a probability distribution 
    torch.nn.SoftMax()
)
torch.nn.init.normal(model[0].weight)
torch.nn.init.normal(model[2].weight)

SyntaxError: invalid syntax (<ipython-input-20-c69bb0a25560>, line 7)

- 为了让我们的模型学会 FizzBuzz 这个游戏，我们需要定义一个损失函数，和一个优化算法。
- 这个有护士算法会不断的优化（降低）损失函数，似的模型在该任务上取得尽可能低得损失值。
- 损失只往往表示我们的模型表现良好，损失值高表示我们的模型表现差
- 由于FizzBuzz游戏本质是一个分类问题，我们选用 cross entropy loss 函数
- 优化函数我们选用 stochastic gradient descent

In [4]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)

以下是模型的训练代码

In [5]:
BATCH_SIZE = 128
for epoch in range(1000):
    for start in range(0,len(train_x),BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = train_x[start:end]
        batchY = train_y[start:end]
        
        if torch.cuda.is_available():
            batchX = batchX.cuda()
            batchY = batchY.cuda()
            
        y_pred = model(batchX) # forward pass
        loss = loss_fn(y_pred,batchY) # counput loss
        print('Epoch: ',epoch," loss: ",loss.item())
        optimizer.zero_grad() # clean gradient
    
        loss.backward() # backward pass       
        optimizer.step() # gradient descent
        

Epoch:  0  loss:  27.872732162475586
Epoch:  0  loss:  28.722644805908203
Epoch:  0  loss:  26.629356384277344
Epoch:  0  loss:  24.986783981323242
Epoch:  0  loss:  22.12517738342285
Epoch:  0  loss:  22.051864624023438
Epoch:  0  loss:  18.95119857788086
Epoch:  0  loss:  15.723039627075195
Epoch:  1  loss:  12.831428527832031
Epoch:  1  loss:  13.59566593170166
Epoch:  1  loss:  10.60025691986084
Epoch:  1  loss:  12.854705810546875
Epoch:  1  loss:  10.27511978149414
Epoch:  1  loss:  11.016809463500977
Epoch:  1  loss:  7.995054721832275
Epoch:  1  loss:  6.968830585479736
Epoch:  2  loss:  7.3105854988098145
Epoch:  2  loss:  8.632174491882324
Epoch:  2  loss:  6.328011989593506
Epoch:  2  loss:  9.077166557312012
Epoch:  2  loss:  7.1867756843566895
Epoch:  2  loss:  7.972975730895996
Epoch:  2  loss:  6.232851505279541
Epoch:  2  loss:  7.187793254852295
Epoch:  3  loss:  6.14825439453125
Epoch:  3  loss:  7.243626594543457
Epoch:  3  loss:  5.606034755706787
Epoch:  3  loss:  

Epoch:  27  loss:  2.4633748531341553
Epoch:  27  loss:  2.2838728427886963
Epoch:  27  loss:  2.083921194076538
Epoch:  28  loss:  2.454561948776245
Epoch:  28  loss:  2.7939767837524414
Epoch:  28  loss:  2.12137508392334
Epoch:  28  loss:  2.5283424854278564
Epoch:  28  loss:  2.1607284545898438
Epoch:  28  loss:  2.442155599594116
Epoch:  28  loss:  2.2624881267547607
Epoch:  28  loss:  2.0500712394714355
Epoch:  29  loss:  2.4275128841400146
Epoch:  29  loss:  2.77685284614563
Epoch:  29  loss:  2.102903127670288
Epoch:  29  loss:  2.4916388988494873
Epoch:  29  loss:  2.1316165924072266
Epoch:  29  loss:  2.4224581718444824
Epoch:  29  loss:  2.242635726928711
Epoch:  29  loss:  2.018228769302368
Epoch:  30  loss:  2.4026482105255127
Epoch:  30  loss:  2.7607181072235107
Epoch:  30  loss:  2.0858285427093506
Epoch:  30  loss:  2.4580602645874023
Epoch:  30  loss:  2.1051511764526367
Epoch:  30  loss:  2.404353618621826
Epoch:  30  loss:  2.224393844604492
Epoch:  30  loss:  1.989

Epoch:  54  loss:  1.5443929433822632
Epoch:  55  loss:  2.0409908294677734
Epoch:  55  loss:  2.4045028686523438
Epoch:  55  loss:  1.8504252433776855
Epoch:  55  loss:  2.032107353210449
Epoch:  55  loss:  1.7771257162094116
Epoch:  55  loss:  2.092750072479248
Epoch:  55  loss:  1.9548240900039673
Epoch:  55  loss:  1.5303479433059692
Epoch:  56  loss:  2.030003547668457
Epoch:  56  loss:  2.3919858932495117
Epoch:  56  loss:  1.8433611392974854
Epoch:  56  loss:  2.022148847579956
Epoch:  56  loss:  1.7699716091156006
Epoch:  56  loss:  2.0826423168182373
Epoch:  56  loss:  1.947178602218628
Epoch:  56  loss:  1.5167412757873535
Epoch:  57  loss:  2.019381284713745
Epoch:  57  loss:  2.3797054290771484
Epoch:  57  loss:  1.8362559080123901
Epoch:  57  loss:  2.012305736541748
Epoch:  57  loss:  1.7630412578582764
Epoch:  57  loss:  2.0726234912872314
Epoch:  57  loss:  1.9397120475769043
Epoch:  57  loss:  1.5024632215499878
Epoch:  58  loss:  2.008945941925049
Epoch:  58  loss:  2

Epoch:  110  loss:  1.5633941888809204
Epoch:  110  loss:  1.707645058631897
Epoch:  110  loss:  1.6650652885437012
Epoch:  110  loss:  1.0588788986206055
Epoch:  111  loss:  1.6351056098937988
Epoch:  111  loss:  1.9046865701675415
Epoch:  111  loss:  1.5724644660949707
Epoch:  111  loss:  1.6724761724472046
Epoch:  111  loss:  1.5606662034988403
Epoch:  111  loss:  1.7033171653747559
Epoch:  111  loss:  1.6617002487182617
Epoch:  111  loss:  1.0555124282836914
Epoch:  112  loss:  1.6306829452514648
Epoch:  112  loss:  1.8987609148025513
Epoch:  112  loss:  1.5690230131149292
Epoch:  112  loss:  1.6685678958892822
Epoch:  112  loss:  1.5579278469085693
Epoch:  112  loss:  1.699082612991333
Epoch:  112  loss:  1.6582791805267334
Epoch:  112  loss:  1.0520597696304321
Epoch:  113  loss:  1.6263442039489746
Epoch:  113  loss:  1.8929917812347412
Epoch:  113  loss:  1.5656508207321167
Epoch:  113  loss:  1.6646937131881714
Epoch:  113  loss:  1.5551109313964844
Epoch:  113  loss:  1.69466

Epoch:  166  loss:  0.9167008399963379
Epoch:  167  loss:  1.4620262384414673
Epoch:  167  loss:  1.6578933000564575
Epoch:  167  loss:  1.4317318201065063
Epoch:  167  loss:  1.5081642866134644
Epoch:  167  loss:  1.4527018070220947
Epoch:  167 1.44225013256073
Epoch:  177  loss:  1.625511884689331
Epoch:  177  loss:  1.4131375551223755
Epoch:  177  loss:  1.4872201681137085
Epoch:  177  loss:  1.4401001930236816
Epoch:  177  loss:  1.5003271102905273
Epoch:  177  loss:  1.4992905855178833
Epoch:  177  loss:  0.8998631238937378
Epoch:  178  loss:  1.4404494762420654
Epoch:  178  loss:  1.6223900318145752
Epoch:  178  loss:  1.4112943410873413
Epoch:  178  loss:  1.4851154088974
Epoch:  178  loss:  1.4388911724090576
Epoch:  178  loss:  1.4979944229125977
Epoch:  178  loss:  1.4972773790359497
Epoch:  178  loss:  0.8984957337379456
Epoch:  179  loss:  1.4385923147201538
Epoch:  179  loss:  1.619340419769287
Epoch:  179  loss:  1.4094891548156738
Epoch:  179  loss:  1.4831289052963257
E

...

Epoch:  728  loss:  0.6322986483573914
Epoch:  729  loss:  0.9913744330406189
Epoch:  729  loss:  1.1274698972702026
Epoch:  729  loss:  1.094577431678772
Epoch:  729  loss:  1.1510765552520752
Epoch:  729  loss:  1.1151719093322754
Epoch:  729  loss:  1.1445825099945068
Epoch:  729  loss:  1.1925671100616455
Epoch:  729  loss:  0.6321083307266235
Epoch:  730  loss:  0.9909665584564209
Epoch:  730  loss:  1.1271240711212158
Epoch:  730  loss:  1.0945137739181519
Epoch:  730  loss:  1.1508029699325562
Epoch:  730  loss:  1.114905834197998
Epoch:  730  loss:  1.1443105936050415
Epoch:  730  loss:  1.1923515796661377
Epoch:  730  loss:  0.6318520307540894
Epoch:  731  loss:  0.9906283020973206
Epoch:  731  loss:  1.1267791986465454
Epoch:  731  loss:  1.0941112041473389
Epoch:  731  loss:  1.150423288345337
Epoch:  731  loss:  1.1144641637802124
Epoch:  731  loss:  1.143998384475708
Epoch:  731  loss:  1.1922125816345215
Epoch:  731  loss:  0.6316763758659363
Epoch:  732  loss:  0.9902278

Epoch:  788  loss:  0.6187503337860107
Epoch:  789  loss:  0.9708420038223267
Epoch:  789  loss:  1.1078896522521973
Epoch:  789  loss:  1.0810848474502563
Epoch:  789  loss:  1.1330969333648682
Epoch:  789  loss:  1.0962533950805664
Epoch:  789  loss:  1.1273305416107178
Epoch:  789  loss:  1.1802526712417603
Epoch:  789  loss:  0.6185051798820496
Epoch:  790  loss:  0.9704920649528503
Epoch:  790  loss:  1.1075847148895264
Epoch:  790  loss:  1.08082115650177
Epoch:  790  loss:  1.1328884363174438
Epoch:  790  loss:  1.0958349704742432
Epoch:  790  loss:  1.1270241737365723
Epoch:  790  loss:  1.180202841758728
Epoch:  790  loss:  0.6182923316955566
Epoch:  791  loss:  0.970129668712616
Epoch:  791  loss:  1.1072721481323242
Epoch:  791  loss:  1.080597996711731
Epoch:  791  1.1778608560562134
Epoch:  802  loss:  0.6156960129737854
Epoch:  803  loss:  0.9657897353172302
Epoch:  803  loss:  1.103066086769104
Epoch:  803  loss:  1.0782073736190796
Epoch:  803  loss:  1.1293483972549438

Epoch:  860  loss:  1.0839635133743286
Epoch:  860  loss:  1.0659900903701782
Epoch:  860  loss:  1.1153392791748047
Epoch:  860  loss:  1.0768141746520996
Epoch:  860  loss:  1.1090962886810303
Epoch:  860  loss:  1.1663905382156372
Epoch:  860  loss:  0.6038951873779297
Epoch:  861  loss:  0.9469568133354187
Epoch:  861  loss:  1.0836628675460815
Epoch:  861  loss:  1.065834641456604
Epoch:  861  loss:  1.1151078939437866
Epoch:  861  loss:  1.076684832572937
Epoch:  861  loss:  1.1088570356369019
Epoch:  861  loss:  1.166219711303711
Epoch:  861  loss:  0.603675365447998
Epoch:  862  loss:  0.946655809879303
Epoch:  862  loss:  1.0834071636199951
Epoch:  862  loss:  1.0656450986862183
Epoch:  862  loss:  1.1149648427963257
Epoch:  862  loss:  1.0764038562774658
Epoch:  862  loss:  1.1085916757583618
Epoch:  862  loss:  1.1660624742507935
Epoch:  862  loss:  0.6033900380134583
Epoch:  863  loss:  0.9467045068740845
Epoch:  863  loss:  1.0831624269485474
Epoch:  863  loss:  1.06528258

Epoch:  918  loss:  0.5930008292198181
Epoch:  919  loss:  0.929634690284729
Epoch:  919  loss:  1.066466212272644
Epoch:  919  loss:  1.0522962808609009
Epoch:  919  loss:  1.102171778678894
Epoch:  919  loss:  1.0622658729553223
Epoch:  919  loss:  1.0956891775131226
Epoch:  919  loss:  1.154323935508728
Epoch:  919  loss:  0.5928528904914856
Epoch:  920  loss:  0.929111897945404
Epoch:  920  loss:  1.066127061843872
Epoch:  920  loss:  1.0522913932800293
Epoch:  920  loss:  1.1019502878189087
Epoch:  920  loss:  1.0620698928833008
Epoch:  920  loss:  1.095455288887024
Epoch:  920  loss:  1.1541719436645508
Epoch:  920  loss:  0.5926553010940552
Epoch:  921  loss:  0.9289741516113281
Epoch:  921  loss:  1.0658879280090332
Epoch:  921  loss:  1.051883578300476
Epoch:  921  loss:  1.1017686128616333
Epoch:  921  loss:  1.0618535280227661
Epoch:  921  loss:  1.0952138900756836
Epoch:  921  loss:  1.1539627313613892
Epoch:  921  loss:  0.5924199223518372
Epoch:  922  loss:  0.92888414859

Epoch:  977  loss:  1.0489693880081177
Epoch:  977  loss:  1.082476258277893
Epoch:  977  loss:  1.1430658102035522
Epoch:  977  loss:  0.5798183083534241
Epoch:  978  loss:  0.9132526516914368
Epoch:  978  loss:  1.050217866897583
Epoch:  978  loss:  1.0390701293945312
Epoch:  978  loss:  1.089938998222351
Epoch:  978  loss:  1.0486817359924316
Epoch:  978  loss:  1.0822588205337524
Epoch:  978  loss:  1.1429855823516846
Epoch:  978 
Epoch:  989  loss:  1.1409921646118164
Epoch:  989  loss:  0.5769357085227966
Epoch:  990  loss:  0.9097437262535095
Epoch:  990  loss:  1.0465741157531738
Epoch:  990  loss:  1.0366417169570923
Epoch:  990  loss:  1.0872840881347656
Epoch:  990  loss:  1.0459253787994385
Epoch:  990  loss:  1.0797607898712158
Epoch:  990  loss:  1.1409387588500977
Epoch:  990  loss:  0.5766522884368896
Epoch:  991  loss:  0.9095937609672546
Epoch:  991  loss:  1.0463294982910156
Epoch:  991  loss:  1.0363596677780151
Epoch:  991  loss:  1.0871014595031738
Epoch:  991  lo

In [ ]:
在测试集上测试训练好的模型，训练集市 1-100 的数字。

In [18]:
testX = torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(1,101)])
if torch.cuda.is_available():
    testX = testX.cuda()
with torch.no_grad():
    test_Y_pred = model(testX)

''' test_Y_pred 是一个 100 * 4 的矩阵，100：数据的维度 4：分类的维度，我们希望取出 最大分类的数值所在的位置
    
    
    test_Y_pred.max(0) 返回值是： 每个分类中 100 个数据 最大的值 ，同时返回了 argmax
    torch.return_types.max(
    values=tensor([8.5216, 7.8722, 4.3155, 3.4836]),
    indices=tensor([23, 76, 27, 88]))
    
    
    test_Y_pred.max(1) 返回值是：  100个数据 中 4个分类中最大的值 ，同时返回了 argmax
    torch.return_types.max(
    values=tensor([  2.0720,  -0.0479,   0.3272,   2.6830,   2.0138,   1.4289,   2.1589,
          5.6255,   5.3915,   4.4673,   3.3578,   6.3112,   6.2853,   5.1545,
          5.7006,  -0.0772,  -0.0865,  -1.4308,  -0.5374,   0.3472,   2.4356,
         -0.7273,   1.3853,   8.5216,   5.0137,   6.2566,   4.0236,   6.7894,
          6.5781,   5.5369,   6.4635,  -8.4202,  -7.3246,  -7.3245,  -4.4900,
         -9.0726,  -6.7353,  -7.3807,  -2.3571,  -3.0759,  -2.9628,  -0.0498,
          0.3855,  -2.5968,  -1.2853,  -1.3854,  -0.5149,  -6.6528,  -9.8269,
         -9.0214,  -7.6572,  -9.6782,  -9.4781,  -9.3804,  -6.2302,  -0.8481,
         -1.5624,  -1.5335,  -0.0819,  -3.0927,  -0.7876,  -1.1919,  -0.1328,
         -0.0144,   3.2060,  -1.7651,   0.8036,  -0.7062,   2.9485,  -0.6380,
          3.4302,   3.8515,   6.8427,   3.7447,   4.0530,   4.3952,   7.8722,
          2.7949,   5.3006,  -2.2080,   0.8530,  -1.4121,  -0.8591,  -1.2939,
          2.5635,  -1.4630,  -0.1757,   5.0776,   5.6624,   3.7090,   3.5376,
          3.5046,   5.2517,   3.4362,   5.3023, -10.3726,  -6.9734,  -8.8782,
         -5.7719, -11.7320]),
    indices=tensor([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
        1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        1, 0, 0, 1]))
    
    因为  test_Y_pred.max(1) 返回的数值有两部分组成，第一部分是数值，第二部分是 argmax ，我们需要的是第二部分所以 test_Y_pred.max(1)【1】
    
    .cpu 是为了将 cuda 中的数值转移如 cpu 中
    .data 是为了取出 tensor 中的数值
    .tolist（）是将数值封装成 list 方便后边的 zip 操作
    
'''
#test_Y_pred.max(1)
predictions = zip(range(1,101),test_Y_pred.max(1)[1].cpu().data.tolist())

print([fizz_buzz_decoding(i,x) for i,x in predictions])

['fizz', '2', '3', '4', '5', 'fizz', '7', '8', 'fizz', '10', '11', 'fizz', '13', 'fizz', '15', 'fizz', 'fizz', '18', '19', 'fizz', 'fizz', '22', '23', '24', '25', '26', '27', '28', '29', 'fizz', '31', '32', 'fizz', '34', '35', '36', '37', 'fizz', '39', 'fizz', 'fizz', 'fizz', '43', 'fizz', 'fizz', 'fizz', '47', '48', '49', '50', '51', '52', '53', 'fizz', '55', '56', '57', '58', '59', 'fizz', '61', 'fizz', '63', '64', 'fizz', '66', '67', 'fizz', 'fizz', '70', 'fizz', '72', 'fizz', '74', 'fizz', 'fizz', 'fizz', '78', 'fizz', 'fizz', 'fizz', '82', 'fizz', 'fizz', 'fizz', 'fizz', 'fizz', '88', '89', '90', '91', 'fizz', 'fizz', 'fizz', '95', '96', 'fizz', '98', '99', 'fizz']
